In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import gym
import argparse
import numpy as np
from ns3gym import ns3env
from DQN_model import DeepQNetwork
from DQN_model import Eval_Model
from DQN_model import Target_Model
import tensorflow as tf

import matplotlib.pyplot as plt

In [2]:
learning_rate = 0.01
reward_decay = 0.9
e_greedy = 0.9
replace_target_iter = 100
memory_size = 500
batch_size = 32
training_episodes = 30
testing_episodes = 10
max_chosen = 3
training_repeat_chosen = []
testing_repeat_chosen = []
episode_time = 300
eval_model_path = '/workspace/model/eval_model'
target_model_path = '/workspace/model/target_model'

In [3]:
#env = gym.make('ns3-v0')
env = ns3env.Ns3Env(debug=True)

#env.reset()

ob_space = env.observation_space
ac_space = env.action_space
ob_space_n = ob_space['slotUsedTable'].shape[0] + ob_space['pktBytes'].shape[0] - 1

print("Observation space: ", ob_space,  ob_space.dtype)
print("Action space: ", ac_space, ac_space.dtype)
print("n_action: ",ac_space.shape[0])

Got new port for ns3gm interface:  9211
Start command:  /tf/ns3-gym/waf --run "tdma-rl --openGymPort=9211 --simSeed=2095172395"
Started ns3 simulation script, Process Id:  5167
Observation space:  Dict(pktBytes:Box(4,), slotUsedTable:Box(32,)) None
Action space:  Box(32,) int64
n_action:  32


In [4]:
try:
    eval_model = tf.keras.models.load_model(eval_model_path)
    target_model = tf.keras.models.load_model(target_model_path)
    print('load model')
except:
    eval_model = Eval_Model(num_actions=ac_space.shape[0])
    target_model = Target_Model(num_actions=ac_space.shape[0])
    print('Create new model')
RL = DeepQNetwork(ac_space.shape[0], max_chosen, ob_space_n,
                  eval_model, target_model, learning_rate, reward_decay, e_greedy, 
                  replace_target_iter, memory_size, batch_size)

load model


In [ ]:
for episode in range(1):
    print ("-----------------------episodes: ", episode, " -----------------------")
    stepIdx = 0
    repeat_chosen_counter = 0
    
    _obs = env.reset()
    queueBytes = _obs[1][0]
    
    _obs = np.array(list(_obs[0]) + list(_obs[1][1:]))
    _obs = np.pad(_obs,(0, ob_space_n - _obs.size), constant_values = 0)

    
    while True:
        stepIdx += 1

        action = np.array(RL.choose_action(_obs, True))

        
        # Using the queuing bytes to decide how many data slot would be choose
        action_num = 0 if queueBytes == 0 else min(max(0, int(queueBytes/6250 - 0.3)) + 1, 3)
        
        action[action_num:] = [-1] * (max_chosen-action_num)
        
        
        #action = env.action_space.sample()
        
        print("---action: ", action)
        print ("Send to env")
        obs, reward, done, info = env.step(action)
        print ("Back!")
        
        if info == "TimeOut":
            print("Step: ", stepIdx)
            print ("Repeat_chosen_counter :", repeat_chosen_counter)
            #RL.learn()
            
            training_repeat_chosen.append(repeat_chosen_counter)
            break
        
        # Get queuing bytes
        queueBytes = obs[1][0]
        
        # Since there are multiple action in one step,
        # according to each action, it would have one reward.
        # But in ns3gym, the reward type is float, it means it can only return one reward,
        # we use the return info to send multiple reward
        
        reward_all = [float(r) for r in info.split(',')]
        info = stepIdx
        
        #print("Step: ", stepIdx)
        #print("---obs, reward, done, info: ", obs, reward_all, done, info)
        
        obs = np.array(list(obs[0]) + list(obs[1][1:]))
        obs = np.pad(obs,(0, ob_space_n - obs.size), constant_values = 0)
        
        for act, r in zip(action,reward_all):
            if act == -1:
                continue
                
            #RL.store_transition(_obs, act, r, obs)
            
            if r <= -100:
                repeat_chosen_counter += 1

                
        if stepIdx % 10000 == 0:
            print("Step: ", stepIdx)
        
        #if (stepIdx > 5000) and (stepIdx % 5 == 0):
        #    RL.learn()
            
        # swap observation
        _obs = obs
        
        
        if done or stepIdx == int(300/0.04*16):
            print("Step: ", stepIdx)
            print ("done")
            print ("Repeat_chosen_counter :", repeat_chosen_counter)
            #RL.learn()
            
            training_repeat_chosen.append(repeat_chosen_counter)
            break
            
        #break
        


-----------------------episodes:  0  -----------------------
Got new port for ns3gm interface:  6191
Start command:  /tf/ns3-gym/waf --run "tdma-rl --openGymPort=6191 --simSeed=3721565489"
Started ns3 simulation script, Process Id:  5339
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
S

---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 

---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 

---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 

---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 

...

Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [ 5 12 -1]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [ 9 31 13]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:

---action:  [28 12 19]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [11 10  8]
Send to env
Back!
---action:  [ 2 20 22]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [16 10 26]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [28 12 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [19 18 24]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [28 

Back!
---action:  [28 12 -1]
Send to env
Back!
---action:  [ 4 21 12]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [28 12 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [28 12 -1]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [ 5 11 19]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [28 -1 -1]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [28 -1 -1]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [28 12 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [28 12 -1]
Send to env
Back!
---action:

---action:  [-1 -1 -1]
Send to env
Back!
---action:  [16 10 29]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [26  4 -1]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [ 8 25 23]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [28 12 -1]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [28 

---action:  [28 12 19]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [28 12 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [28 12 -1]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [29 12  8]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [28 12 -1]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [28 12 -1]
Send to env
Back!
---action:  [28 12 19]
Send to env
Back!
---action:  [-1 -1 -1]
Send to env
Back!
---action:  [28 

In [ ]:
for episode in range(1):
    print ("-----------------------episodes: ", episode, " -----------------------")
    stepIdx = 0
    repeat_chosen_counter = 0
    
    _obs = env.reset()
    queueBytes = _obs[1][0]
    
    _obs = np.array(list(_obs[0]) + list(_obs[1][1:]))
    _obs = np.pad(_obs,(0, ob_space_n - _obs.size), constant_values = 0)

    
    while True:
        stepIdx += 1

        action = np.array(RL.choose_action(_obs, False))

        
        # Using the queuing bytes to decide how many data slot would be choose
        action_num = 0 if queueBytes == 0 else min(max(0, int(queueBytes/6250 - 0.3)) + 1, 3)
        
        action[action_num:] = [-1] * (max_chosen-action_num)
        
        
        #action = env.action_space.sample()
        
        #print("---action: ", action)
        #print ("Send to env")
        obs, reward, done, info = env.step(action)
        #print ("Back!")
        
        if info == "TimeOut":
            print("Step: ", stepIdx)
            print ("Repeat_chosen_counter :", repeat_chosen_counter)
            #RL.learn()
            
            testing_repeat_chosen.append(repeat_chosen_counter)
            break
        
        # Get queuing bytes
        queueBytes = obs[1][0]
        
        # Since there are multiple action in one step,
        # according to each action, it would have one reward.
        # But in ns3gym, the reward type is float, it means it can only return one reward,
        # we use the return info to send multiple reward
        
        reward_all = [float(r) for r in info.split(',')]
        info = stepIdx
        
        #print("Step: ", stepIdx)
        #print("---obs, reward, done, info: ", obs, reward_all, done, info)
        
        obs = np.array(list(obs[0]) + list(obs[1][1:]))
        obs = np.pad(obs,(0, ob_space_n - obs.size), constant_values = 0)
        
        for act, r in zip(action,reward_all):
            if act == -1:
                continue
                
            #RL.store_transition(_obs, act, r, obs)
            
            if r <= -100:
                repeat_chosen_counter += 1
        
        #if (stepIdx > 64*50) and (stepIdx % 5 == 0):
        #    RL.learn()
            
        # swap observation
        _obs = obs
        
        
        if done or stepIdx == int(300/0.04*16):
            print("Step: ", stepIdx)
            print ("done")
            print ("Repeat_chosen_counter :", repeat_chosen_counter)
            #RL.learn()
            
            testing_repeat_chosen.append(repeat_chosen_counter)
            break
            
        #break
        


In [ ]:
# step counter of each episode
plt.plot(training_repeat_chosen)
plt.show()

In [ ]:
RL.plot_cost()

In [ ]:
env.close()

In [ ]:
eval_model.save('/workspace/model/eval_model', save_format='tf')
target_model.save('/workspace/model/target_model', save_format='tf')